In [1]:
import pandas as pd
import torch
from huggingface_hub import login
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [2]:
# TODO login  to huggingface_hub

In [3]:
def load_dataset():  # -> pd.DataFrame:
    """
    Wczytuje dane z arkusza Google Sheets, usuwa nadmiarowe kolumny i filtruje wiersze z "prawidłowe" == "T".

    Returns:
        pd.DataFrame: Odfiltrowana ramka danych.
    """
    # URL do arkusza Google Sheets w formacie CSV
    sheet_url = "https://docs.google.com/spreadsheets/d/1-Ag5DOHUywOeg_lwDi8ymRsREmHZ87s6d6FUPfLr8tc/export?format=csv"
    
    # Wczytanie arkusza
    df = pd.read_csv(sheet_url)

    # Usunięcie nadmiarowych kolumn (np. 'Unnamed')
    df.drop(columns=df.columns[df.columns.str.contains('^Unnamed')], inplace=True)

    # Filtrowanie wierszy, gdzie "Prawidłowe" == "T"
    if "Prawidłowe" in df.columns:
        df = df.loc[df["Prawidłowe"] == "T"]
    else:
        raise ValueError("Brak wymaganej kolumny 'prawidłowe' w ramce danych.")
    
    return df

In [4]:
df = load_dataset()
df.head()

,Pytanie,Odpowiedź,link do testu w B2,rodzaj zagadki,Zaskakujące odpowiedzi Bielika,Prawidłowe
0,Kto ma imieniny 30 lutego?,"Nikt, bo nie ma takiego dnia.",NaN,NaN,NaN,T
1,Ile przekątnych ma trójkąt?,"Zero, trójkąt nie ma przekątnych",NaN,sprzeczność z uwagi na definicję pojęcia,NaN,T
2,W którym roku wydarzyła się na terenie Polski ...,Nie było takiej bitwy.,NaN,NaN,NaN,T
3,W którym miesiącu zaczyna się 5 kwartał roku?,Rok ma tylko 4 kwartały.,NaN,NaN,NaN,T
4,Ile skrzydeł ma pies?,Zero. Psy nie mają skrzydeł.,NaN,NaN,NaN,T


In [5]:
def get_answer():
    for question in df['Pytanie']:
       answer = 'example'
       return answer

In [6]:
df["Odpowiedź:"] = get_answer()

In [7]:
df.head()

,Pytanie,Odpowiedź,link do testu w B2,rodzaj zagadki,Zaskakujące odpowiedzi Bielika,Prawidłowe,Odpowiedź:
0,Kto ma imieniny 30 lutego?,"Nikt, bo nie ma takiego dnia.",NaN,NaN,NaN,T,example
1,Ile przekątnych ma trójkąt?,"Zero, trójkąt nie ma przekątnych",NaN,sprzeczność z uwagi na definicję pojęcia,NaN,T,example
2,W którym roku wydarzyła się na terenie Polski ...,Nie było takiej bitwy.,NaN,NaN,NaN,T,example
3,W którym miesiącu zaczyna się 5 kwartał roku?,Rok ma tylko 4 kwartały.,NaN,NaN,NaN,T,example
4,Ile skrzydeł ma pies?,Zero. Psy nie mają skrzydeł.,NaN,NaN,NaN,T,example


In [8]:
# wczytanie modelu 
model_id = "speakleash/Bielik-7B-Instruct-v0.1"

# kwantyzacja modelu
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quantization_config
)

# Wczytanie tokenizer z przypisaniem pad_token_id
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token  # Ustawienie pad_token_id jako eos_token_id

pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [9]:
pipe("Ile jest procent cukru w cukrze?",
     max_length=100,
     # temperature=0.6,
     # top_k=10
     )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\ankaz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\bitsandbytes\nn\modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


[{'generated_text': 'Ile jest procent cukru w cukrze?\nTo pytanie z serii absurdów na temat produktów spożywczych. Ile procent cukru jest w cukrze? Raczej nie ma co się zastanawiać - 100%. A właściwie, patrząc z prawnego punktu widzenia, producent cukru nie musi podawa'}]